## Read graph of metafeatures

In [1]:
import networkx as nx
g_x = nx.read_gpickle("encoded_fasttext.gpickle")
#g = nx.read_gpickle("encoded_features.gpickle")
#g = nx.read_gpickle("siimple.gpickle")
order = 0
for x,n in sorted(g_x.nodes(data=True)):
    t = n['tipo']
    if t == "dataset":
        n['tipo'] = 0
    if t == "feature dataset":
        n['tipo'] = 1
    if t == "literal dataset":
        n['tipo'] = 2
    if t == "attribute":
        n['tipo'] = 3
    if t == "feature attribute":
        n['tipo'] = 4
    if t == "literal attribute":
        n['tipo'] = 5  
    n['order']=order
    order+=1

In [2]:
datasets = [x for x,y in g_x.nodes(data=True) if y['tipo']==0]
order = [y['order'] for x,y in g_x.nodes(data=True) if y['tipo']==0]

In [3]:
map_order = dict(zip(datasets,order))

In [4]:
map_order['DS_1']

374

## Get 60% train, 20% val and 20% test from csv

In [5]:
import numpy as np
import pandas as pd

def read_dataset(path,drop_columns=None,keep_columns=None):
    #get rid of useless columns
    csv_data = pd.read_csv(path)
    
    if keep_columns != None:
        #keep only these columns
        return csv_data.filter(items=keep_columns)
    
    if drop_columns!= None:
        #drop these and keep the rest
        return csv_data.drop(drop_columns, axis=1)
    
    #finally, didn't drop or filter any column
    return csv_data     


In [6]:
from random import randrange
def get_samples(df):
    train_mask = []
    val_mask = []
    test_mask = []
    for index, row in df.iterrows():
        if row[2] == 1:
            if randrange(0, 10) <= 7:
                train_mask.append([row[0],row[1],row[2]])
            else:
                test_mask.append([map_order['DS_'+str(row[0])],map_order['DS_'+str(row[1])],row[2]])
#                 if index % 2 == 0:
#                     val_mask.append([map_order['DS_'+str(row[0])],map_order['DS_'+str(row[1])],row[2]])
#                 else:
#                     test_mask.append([map_order['DS_'+str(row[0])],map_order['DS_'+str(row[1])],row[2]])
        else:
            if randrange(0, 10) > 8:
#                 if randrange(0, 30) <= 21:
                if randrange(0, 30) <= 24:
                    train_mask.append([map_order['DS_'+str(row[0])],map_order['DS_'+str(row[1])],row[2]])
                else:
                    test_mask.append([map_order['DS_'+str(row[0])],map_order['DS_'+str(row[1])],row[2]])
#                     if index % 2 == 0:
#                         val_mask.append([map_order['DS_'+str(row[0])],map_order['DS_'+str(row[1])],row[2]])
#                     else:
#                         test_mask.append([map_order['DS_'+str(row[0])],map_order['DS_'+str(row[1])],row[2]])
    return train_mask,test_mask
#     return train_mask,val_mask,test_mask

In [7]:
df_matching = read_dataset("./openml_203ds_datasets_matching.csv",keep_columns=["'dataset1_id'", "'dataset2_id'","'matching_topic'"]);
# train_mask,val_mask,test_mask = get_samples(df_matching)
# print("Train samples: "+str(len(train_mask)) + " Val samples: "+str(len(val_mask))+ " Test samples: "+str(len(test_mask)))
train_mask,test_mask = get_samples(df_matching)
print("Train samples: "+str(len(train_mask)) + " Test samples: "+str(len(test_mask)))


Train samples: 2053 Test samples: 451


In [8]:
test_mask

[[374, 471, 0],
 [374, 509, 0],
 [374, 449, 0],
 [374, 377, 0],
 [374, 529, 0],
 [571, 523, 0],
 [571, 474, 0],
 [571, 568, 0],
 [576, 480, 1],
 [576, 487, 0],
 [576, 547, 1],
 [576, 563, 0],
 [576, 574, 1],
 [576, 429, 0],
 [576, 512, 1],
 [417, 570, 0],
 [417, 510, 0],
 [417, 450, 0],
 [417, 479, 1],
 [417, 573, 0],
 [418, 382, 1],
 [418, 451, 0],
 [464, 469, 0],
 [464, 535, 0],
 [464, 382, 1],
 [464, 426, 0],
 [464, 446, 0],
 [464, 450, 0],
 [464, 494, 0],
 [464, 557, 0],
 [466, 564, 0],
 [466, 398, 0],
 [466, 494, 0],
 [470, 508, 1],
 [470, 401, 0],
 [470, 444, 0],
 [470, 376, 0],
 [470, 559, 0],
 [476, 502, 0],
 [476, 510, 1],
 [476, 397, 0],
 [476, 408, 0],
 [476, 450, 0],
 [476, 461, 0],
 [486, 389, 0],
 [486, 407, 0],
 [486, 456, 0],
 [491, 414, 0],
 [491, 461, 1],
 [491, 511, 1],
 [495, 527, 0],
 [496, 508, 1],
 [496, 469, 0],
 [496, 538, 0],
 [496, 475, 0],
 [499, 524, 0],
 [499, 540, 0],
 [499, 430, 0],
 [499, 463, 0],
 [499, 436, 0],
 [499, 555, 0],
 [501, 508, 0],
 [501, 4

## Deep graph library

In [9]:
import dgl
#convert from networkx to graph deep library format
g = dgl.DGLGraph()
#gdl.from_networkx(g,['vector'])
g.from_networkx(g_x,node_attrs=['tipo','vector'], edge_attrs=None)

Using backend: pytorch


In [10]:
import dgl
import dgl.function as fn
import torch as th
import torch.nn as nn
import torch.nn.functional as F
from dgl import DGLGraph

gcn_msg = fn.copy_src(src='vector', out='m')
gcn_reduce = fn.sum(msg='m', out='vector')

In [11]:
class GCNLayer(nn.Module):
    def __init__(self, in_feats, out_feats):
        super(GCNLayer, self).__init__()
        self.linear = nn.Linear(in_feats, out_feats)

    def forward(self, g,feature):
        # Creating a local scope so that all the stored ndata and edata
        # (such as the `'h'` ndata below) are automatically popped out
        # when the scope exits.
        with g.local_scope():
            g.ndata['vector'] = feature
            g.update_all(gcn_msg, gcn_reduce)
            h = g.ndata['vector']
            return self.linear(h)

In [29]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.layer1 = GCNLayer(300, 300)
        self.layer2 = GCNLayer(300, 300)
    
    def forward(self, g,features):
        x = F.relu(self.layer1(g, features))
        x = F.relu(self.layer2(g, x))
        return x
net = Net()
print(net)

Net(
  (layer1): GCNLayer(
    (linear): Linear(in_features=300, out_features=300, bias=True)
  )
  (layer2): GCNLayer(
    (linear): Linear(in_features=300, out_features=300, bias=True)
  )
)


In [30]:
from scipy.spatial.distance import cosine

def cosine_vectors(vec1,vec2):
    return (1- cosine(vec1,vec2))

def resultSet_train(features,mask):
    indices = []
    labels = []
    for n in mask:
        resultCos = cosine(features[n[0]],features[n[1]])
        # index 0 is not similar, index 1 is similar
        result = th.tensor([1 - resultCos , resultCos])
        indices.append(result)
        labels.append(n[2])
    return indices,labels


def resultSet_eval(features,mask):
    indices = []
    labels = []
    for n in mask:
        resultCos = cosine(features[n[0]],features[n[1]])
        # index 0 is not similar, index 1 is similar
        result = th.tensor([1 - resultCos , resultCos])
        # get the index that is greater
        out = th.max(result, dim=1)
        indices.append(out)
        labels.append(n[2])
    return indices,labels

def evaluate(model, g, features, mask):
    model.eval()
    with th.no_grad():
        embeddings = model(g, features)
        indices , labels = resultSet_eval(embeddings,mask)
        correct = th.sum(indices == labels)
        return correct.item() * 1.0 / len(labels)

In [31]:
net = Net()
net.train()
embeddings = net(g, g.ndata['vector'])
indices,labels = resultSet_train(embeddings,train_mask)

RuntimeError: Can't call numpy() on Variable that requires grad. Use var.detach().numpy() instead.

In [27]:
indices

[tensor([0.9978, 0.0022]),
 tensor([0.9726, 0.0274]),
 tensor([0.9938, 0.0062]),
 tensor([0.9177, 0.0823]),
 tensor([0.9965, 0.0035]),
 tensor([0.9658, 0.0342]),
 tensor([0.9671, 0.0329]),
 tensor([0.9662, 0.0338]),
 tensor([0.9933, 0.0067]),
 tensor([0.9628, 0.0372]),
 tensor([0.9619, 0.0381]),
 tensor([0.8512, 0.1488]),
 tensor([0.9957, 0.0043]),
 tensor([0.9365, 0.0635]),
 tensor([0.9041, 0.0959]),
 tensor([0.9858, 0.0142]),
 tensor([0.8681, 0.1319]),
 tensor([0.9227, 0.0773]),
 tensor([0.8892, 0.1108]),
 tensor([0.9982, 0.0018]),
 tensor([0.9416, 0.0584]),
 tensor([0.9025, 0.0975]),
 tensor([0.9673, 0.0327]),
 tensor([0.9802, 0.0198]),
 tensor([0.9594, 0.0406]),
 tensor([0.9975, 0.0025]),
 tensor([0.9669, 0.0331]),
 tensor([0.9846, 0.0154]),
 tensor([0.9989, 0.0011]),
 tensor([0.9759, 0.0241]),
 tensor([0.9572, 0.0428]),
 tensor([0.9988, 0.0012]),
 tensor([0.9682, 0.0318]),
 tensor([0.9961, 0.0039]),
 tensor([0.9281, 0.0719]),
 tensor([0.9710, 0.0290]),
 tensor([0.9235, 0.0765]),
 

In [ ]:
import time
import numpy as np
net = Net()
#g, features, labels, train_mask, test_mask

optimizer = th.optim.Adam(net.parameters(), lr=1e-2)
dur = []
for epoch in range(5):
    if epoch >=3:
        t0 = time.time()

    net.train()
    embeddings = net(g, g.ndata['vector'])
    indices,labels = resultSet_train(embeddings,train_mask)
    loss = F.nll_loss(indices, labels)
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    if epoch >=3:
        dur.append(time.time() - t0)
    
    acc = evaluate(net, g, features, labels, test_mask)
    print("Epoch {:05d} | Loss {:.4f} | Test Acc {:.4f} | Time(s) {:.4f}".format(
            epoch, loss.item(), acc, np.mean(dur)))

## Tests

In [ ]:
#get data from a specific node
g.nodes[0].data
#get data from nodes
g.ndata
#another way of accessing data from a node
g.ndata['tipo'][0]

In [ ]:
g.edges()

In [ ]:
g.has_edge_between(374,17619)

In [ ]:
g.ndata

In [ ]:
for n in g.nodes:
    print (n.data['tipo'])